In [1]:
from curses import init_color
import os
import os.path as osp
import copy
import sys
import time 
from datetime import timedelta
from datetime import datetime
import random
import time
from tqdm import tqdm
from collections import Counter

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import argparse
import pickle
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from models.AutoEncoder import AutoEncoder_RGB, AutoEncoder_Depth
from models.AutoEncoder import AutoEncoder_Intergrated_Basic, AutoEncoder_Intergrated_Proposed
from models.Network import Face_Detection_Model
from dataloader.dataloader import load_dataset, load_test_dataset
from utility import draw_train_and_test_loss, draw_train_and_test_loss1, draw_accuracy_and_f1_during_training, draw_accuracy_and_f1_during_training1

print(torch.__version__,torch.cuda.is_available(),torch.cuda.device_count())
copy_before_model = False
server = 'server6'
print(f'server : {server} , copy_before_model : {copy_before_model}')

1.12.1+cu113 True 2
server : server6 , copy_before_model : False


In [2]:
# def seed_worker(worker_id):
#     worker_seed = torch.initial_seed() % 2**32
#     np.random.seed(worker_seed)
#     random.seed(worker_seed)

def booltype(str):
    if isinstance(str, bool):
        return str
    if str.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif str.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentError("Boolean value expected")

# def model_save(model, epoch, optimizer, train_loss, val_loss, train_f1, valid_f1, path) :
#     torch.save({
#                 'model_state_dict': model.state_dict(),
#                 'epoch': epoch,
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'train_loss': train_loss,
#                 'val_loss' : val_loss,
#                 'train_f1' : train_f1,
#                 'val_f1' : valid_f1
#                 }, path)
#     print('Model Save ! > ', path)

In [3]:
# args option
parser = argparse.ArgumentParser(description='face anto-spoofing')
parser.add_argument('--batchsize', default=4, type=int, help='batch size')
parser.add_argument('--workers', default=4, type=int, help='number of workers, 4')
parser.add_argument('--epochs', default=100, type=int, help='train epochs')        
parser.add_argument('--trainratio', default=1.0, type=float, help='ratio to divide train dataset')                               
parser.add_argument('--lr', default=1e-3, type=float, help='learning rate(default: 0.001)')
parser.add_argument('--skf', default=0, type=int, help='stratified k-fold')
parser.add_argument('--attacktype', default='prm', type=str, help='Kinds of Presentation Attacks: r, p, m, prm')
parser.add_argument('--dataset', default=12, type=int, help='dataset type: 12 or 15')
parser.add_argument('--model', default='', type=str, help='rgb, rgbd, rgbp, rgbdp')   
parser.add_argument('--inputchannel', default=3, type=int, help='inputchannel')
parser.add_argument('--crop', default=False, type=booltype, help='use crop (default: False)')
parser.add_argument('--ae-path', default='', type=str, help='Pretrained AutoEncoder path')
parser.add_argument('--save-path', default='./hm_save_file', type=str, help='train logs path')
parser.add_argument('--model-path', default='', type=str, help='model parameter path')
# parser.add_argument('--init-path', default='', type=str, help='model init parameter path')
# parser.add_argument('--message', default='', type=str, help='parameter file name')                     
parser.add_argument('--seed', default=1, type=int, help='Seed for random number generator')
parser.add_argument('--cuda', default=0, type=int, help='gpu number')                                         
parser.add_argument('--device', default='', type=str, help='device when cuda is available')                       
args = parser.parse_known_args()[0]

In [4]:
def fix_seed(random_seed):
    """
    fix seed to control any randomness from a code 
    (enable stability of the experiments' results.)
    """
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
#     torch.use_deterministic_algorithms(True)
    np.random.seed(random_seed)
    random.seed(random_seed)
    os.environ['PYTHONHASHSEED'] = str(random_seed)
#     os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    
fix_seed(args.seed)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

In [5]:

def train(args, train_loader, test_loader, outdoor_loader, dark_loader):
    
    # Model 생성 및 아키텍쳐 출력   
    model = Face_Detection_Model(args.inputchannel).to(args.device)    
    # original = model.parameters()
    
    # ckpt = torch.load(args.init_path)
    # model.load_state_dict(ckpt['model_state_dict'])
    # init = model.parameters()
    
    # a, b = [], []
    # for param in original:
    #     a.append(param)
    # for param in init:
    #     b.append(param)
    # for i in range(len(a)):
    #     compare = torch.eq(a[i], b[i])
    #     # print(compare)
    #     # print(torch.any(compare))  
    #     print(f"{a[i]} {b[i]}")
    
    # Loss, 옵티마이저, 스케줄러 생성 
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    # scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)
        
    # Train 
    train_performs = {'ACC':[],'F1':[]}
    test_performs = {'ACC':[],'F1':[], 'Info':[]}
    outdoor_performs = {'ACC':[],'F1':[], 'Info':[]}
    dark_performs = {'ACC':[],'F1':[], 'Info':[]}

    # Variables 
    total_train_loss = []
    total_test_loss = []
    total_outdoor_loss = []
    total_dark_loss = []
    
    best_test_f1 = 0
    best_test_epoch = 0   
    best_test_accu = 0 
    
    epochs = args.epochs + 1
    sigmoid = nn.Sigmoid()
    loss_fn = nn.BCEWithLogitsLoss()

    # model_save(model, 0, optimizer, 0, 0, 0 , 0, osp.join(args.model_path, f"init_parameter.pth")) 
    
    fisrt_init_path = './hm_save_file/server1_beforetrain.pth'
    if server == 'server1' :
        if copy_before_model :
            global before_model
            before_model = copy.deepcopy(model)
            
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        },fisrt_init_path)
    elif server == 'server6' :
        global check_model
            
        checkpoint = torch.load(fisrt_init_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        check_model = copy.deepcopy(model)
    
    
    indoor_check = {'train_loss':[],'test_loss':[],'train_f1':[],'test_f1':[],'train_acc':[],'test_acc':[]}
    outdoor_check  = {'test_loss':[],'test_f1':[],'test_acc':[]}
    dark_check = {'test_loss':[],'test_f1':[],'test_acc':[]}

    for epoch in range(1, epochs) :
        model.train()
        train_loss = []
        train_probs, train_labels = [],[]
        train_bar = tqdm(enumerate(train_loader))
        for step, data in train_bar :  
            rgb, cloud, depth, label = data
            rgb = rgb.float().to(args.device)
            cloud = cloud.float().to(args.device)
            depth = depth.float().to(args.device)
            label = label.float().to(args.device)

            features = rgb
            if args.model == "rgbd":
                features = torch.cat([rgb, depth], dim=1)
            elif args.model == "rgbp":
                features = torch.cat([rgb, cloud], dim=1)
            elif args.model == "rgbdp":
                features = torch.cat([rgb, depth, cloud], dim=1)  
     
            optimizer.zero_grad()
            logits, _ = model(features)
            
            logits = logits[:,0]  # logtit (4.2) 텐서이던데 첫째는 batch 이겠고. 근데 왜 0번째 값만 쓸까?
            loss = loss_fn(logits, label.float())
            loss.backward()
            optimizer.step()
            
            probs = sigmoid(logits)
            train_loss.append(loss.item())
            train_probs += probs.cpu().detach().tolist()
            train_labels += label.cpu().detach().tolist()
            
            train_bar.set_description("[Train] Epoch[{}/{}][{}/{}] Loss:{} Loss(mean):{}".format(epoch,epochs-1,step+1,len(train_loader),
                                                        round(loss.item(),5),round(np.array(train_loss).mean(),10)
                                                                ))
        
        train_loss_mean = round(np.array(train_loss).mean(), 10)        
        
        # scheduler.step()
        print("[Train] Epoch[{}/{}][{}/{}] Loss:{} Loss(mean):{}".format(epoch,epochs-1,step+1,len(train_loader),
                                                        round(loss.item(),5),train_loss_mean))    
        total_train_loss.append(train_loss_mean)
        train_acc = accuracy_score(np.array(train_labels), np.round(train_probs))
        train_f1 = f1_score(np.array(train_labels), np.round(train_probs), average='macro')
        print(f'Train Accuracy : {train_acc:.4f}')
        print(f'Train F1-score : {train_f1:.4f}')
        train_performs['ACC'].append(train_acc)
        train_performs['F1'].append(train_f1)
        print(f'  > Counter(train_labels) : {Counter(train_labels)}')
        print(f'  > Counter(train_preds) : {Counter(np.round(train_probs))}')
        cf = confusion_matrix(np.array(train_labels), np.round(train_probs))
        cf = pd.DataFrame(cf)
        cf.columns = ['Predicted:0','Predicted:1']
        cf.index = ['Label:0','Label:1']    
        print(' --- [Train] Confustion_Matrix & Classification_Report --- ')
        
        # display(cf)
        print(cf.to_string())
        report = classification_report(np.array(train_labels), np.round(train_probs))
        print(report)
        
        indoor_check['train_loss'].append(np.array(train_loss).mean())
        indoor_check['train_f1'].append(train_f1)
        indoor_check['train_acc'].append(train_acc)
            
        model.eval()
        test_loss = []
        test_probs, test_labels = [],[]
        
        # Indoor 
        test_bar = tqdm(enumerate(test_loader))
        for step, data in test_bar :  
            rgb, cloud, depth, label = data
            rgb = rgb.float().to(args.device)
            cloud = cloud.float().to(args.device)
            depth = depth.float().to(args.device)
            label = label.float().to(args.device)
            
            features = rgb
            if args.model == "rgbd":
                features = torch.cat([rgb, depth], dim=1)
            elif args.model == "rgbp":
                features = torch.cat([rgb, cloud], dim=1)
            elif args.model == "rgbdp":
                features = torch.cat([rgb, depth, cloud], dim=1)  

            logits,_ = model(features)
            logits = logits[:,0]
            loss = loss_fn(logits, label.float())
            
            probs = sigmoid(logits)
            test_loss.append(loss.item())
            test_probs += probs.cpu().detach().tolist()
            test_labels += label.cpu().detach().tolist()        
            test_bar.set_description("[Test] Epoch[{}/{}][{}/{}] Loss:{} Loss(mean):{}".format(epoch,epochs-1,step+1,len(test_loader),
                                                        round(loss.item(),5),round(np.array(test_loss).mean(),5)
                                                                ))
            
        indoor_check['test_loss'].append(np.array(test_loss).mean())
        indoor_check['test_f1'].append(f1_score(np.array(test_labels), np.round(test_probs), average='macro'))
        indoor_check['test_acc'].append(accuracy_score(np.array(test_labels), np.round(test_probs)))
            
        outdoor_loss = []
        outdoor_probs, outdoor_labels = [], []
            
        outdoor_bar = tqdm(enumerate(outdoor_loader))
        for step, data in outdoor_bar :  
            rgb, cloud, depth, label = data
            rgb = rgb.float().to(args.device)
            cloud = cloud.float().to(args.device)
            depth = depth.float().to(args.device)
            label = label.float().to(args.device)
            
            features = rgb
            if args.model == "rgbd":
                features = torch.cat([rgb, depth], dim=1)
            elif args.model == "rgbp":
                features = torch.cat([rgb, cloud], dim=1)
            elif args.model == "rgbdp":
                features = torch.cat([rgb, depth, cloud], dim=1)  

            logits,_ = model(features)
            logits = logits[:,0]
            loss = loss_fn(logits, label.float())
            
            probs = sigmoid(logits)
            outdoor_loss.append(loss.item())
            outdoor_probs += probs.cpu().detach().tolist()
            outdoor_labels += label.cpu().detach().tolist()        
            outdoor_bar.set_description("[Test] Epoch[{}/{}][{}/{}] Loss:{} Loss(mean):{}".format(epoch,epochs-1,step+1,len(test_loader),
                                                        round(loss.item(),5),round(np.array(outdoor_loss).mean(),5)
                                                                ))
        
        outdoor_check['test_loss'].append(np.array(outdoor_loss).mean())
        outdoor_check['test_f1'].append(f1_score(np.array(outdoor_labels), np.round(outdoor_probs), average='macro'))
        outdoor_check['test_acc'].append(accuracy_score(np.array(outdoor_labels), np.round(outdoor_probs)))
          
        dark_loss = []
        dark_probs, dark_labels = [], []  
          
        dark_bar = tqdm(enumerate(dark_loader))
        for step, data in dark_bar :  
            rgb, cloud, depth, label = data
            rgb = rgb.float().to(args.device)
            cloud = cloud.float().to(args.device)
            depth = depth.float().to(args.device)
            label = label.float().to(args.device)
            
            features = rgb
            if args.model == "rgbd":
                features = torch.cat([rgb, depth], dim=1)
            elif args.model == "rgbp":
                features = torch.cat([rgb, cloud], dim=1)
            elif args.model == "rgbdp":
                features = torch.cat([rgb, depth, cloud], dim=1)  

            logits,_ = model(features)
            logits = logits[:,0]
            loss = loss_fn(logits, label.float())
            
            probs = sigmoid(logits)
            dark_loss.append(loss.item())
            dark_probs += probs.cpu().detach().tolist()
            dark_labels += label.cpu().detach().tolist()        
            dark_bar.set_description("[Test] Epoch[{}/{}][{}/{}] Loss:{} Loss(mean):{}".format(epoch,epochs-1,step+1,len(test_loader),
                                                        round(loss.item(),5),round(np.array(dark_loss).mean(),5)
                                                                ))
                
        dark_check['test_loss'].append(np.array(dark_loss).mean())
        dark_check['test_f1'].append(f1_score(np.array(dark_labels), np.round(dark_probs), average='macro'))
        dark_check['test_acc'].append(accuracy_score(np.array(dark_labels), np.round(dark_probs)))          
            
        print("[Test] Epoch[{}/{}][{}/{}] Loss:{} Loss(mean):{}".format(epoch,epochs-1,step+1,len(test_loader),
                                                        round(loss.item(),5),round(np.array(test_loss).mean(),5)))
        test_loss_mean = round(np.array(test_loss).mean(),10)
        outdoor_loss_mean = round(np.array(outdoor_loss).mean(),10)
        dark_loss_mean = round(np.array(dark_loss).mean(),10)
        total_test_loss.append(test_loss_mean)
        total_outdoor_loss.append(outdoor_loss_mean)
        total_dark_loss.append(dark_loss_mean)
        test_acc = accuracy_score(np.array(test_labels), np.round(test_probs))
        test_f1 = f1_score(np.array(test_labels), np.round(test_probs), average='macro')
        outdoor_acc = accuracy_score(np.array(outdoor_labels), np.round(outdoor_probs))
        outdoor_f1 = f1_score(np.array(outdoor_labels), np.round(outdoor_probs), average='macro')
        dark_acc = accuracy_score(np.array(dark_labels), np.round(dark_probs))
        dark_f1 = f1_score(np.array(dark_labels), np.round(dark_probs), average='macro')        
        
        print(f'Test Accuracy : {test_acc:.4f}')
        print(f'Test F1-score : {test_f1:.4f}')
        test_performs['ACC'].append(test_acc)
        test_performs['F1'].append(test_f1)
        outdoor_performs['ACC'].append(outdoor_acc)
        outdoor_performs['F1'].append(outdoor_f1)
        dark_performs['ACC'].append(dark_acc)
        dark_performs['F1'].append(dark_f1)
        # logger.Print(f'  > Counter(test_labels) : {Counter(test_labels)}')
        # logger.Print(f'  > Counter(test_probs) : {Counter(np.round(test_probs))}')
        test_cf = confusion_matrix(np.array(test_labels), np.round(test_probs))
        test_cf = pd.DataFrame(test_cf)
        test_cf.columns = ['Predicted:0','Predicted:1']
        test_cf.index = ['Label:0','Label:1']    
        print(' --- [Test] Confustion_Matrix & Classification_Report --- ')
        # display(test_cf)
        print(test_cf.to_string())
        test_performs['Info'].append(test_cf.to_string())
        test_report = classification_report(np.array(test_labels), np.round(test_probs))
        print(test_report)   
        
        outdoor_acc = accuracy_score(np.array(dark_labels), np.round(dark_probs))
        outdoor_F1 = f1_score(np.array(dark_labels), np.round(dark_probs), average='macro')
        outdoor_l = np.array(dark_loss).mean()
                
        dark_acc = accuracy_score(np.array(dark_labels), np.round(dark_probs))
        dark_F1 = f1_score(np.array(dark_labels), np.round(dark_probs), average='macro')
        dark_l = np.array(dark_loss).mean()
        
        # 모두 다 저장 
        print(' @@ New Save Situation !! @@ ')  
        print(f' @@ New Current Epoch : {epoch}')    
        print(f' ')
        print(f' @@ New Train Accuracy : {train_acc:.4f}')
        print(f' @@ New Train F1-Score : {train_f1:.4f}')
        print(f' ')
        print(f' @@ New Indoor Accuracy : {test_acc:.4f}')
        print(f' @@ New Indoor F1-Score : {test_f1:.4f}')
        print(f' ')
        print(f' @@ New Outdoor Accuracy : {outdoor_acc:.4f}')
        print(f' @@ New Outdoor F1-Score : {outdoor_F1:.4f}')
        print(f' ')
        print(f' @@ New Dark Accuracy : {dark_acc:.4f}')
        print(f' @@ New Dark F1-Score : {dark_F1:.4f}')
        print(f' ')
        print(f' @@ New Train Loss : {train_loss_mean:}') 
        print(f' @@ New Indoor Loss : {np.array(test_loss).mean():}') 
        print(f' @@ New Outdoor Loss : {outdoor_l}')
        print(f' @@ New Dark Loss : {dark_l}')
        print(f' ')
        print(f'Saving Model ..... ')
                    
#         model_save(model,epoch,optimizer,np.array(train_loss).mean(),np.array(test_loss).mean(),
#                 train_f1,test_f1,
#                 osp.join(args.model_path, f"epoch_{epoch}_model"+'.pth'))            
                
    draw_train_and_test_loss(args, total_train_loss, total_test_loss)
    draw_accuracy_and_f1_during_training(args, test_performs["ACC"], test_performs["F1"])
    draw_train_and_test_loss1(args, total_train_loss, total_test_loss, total_outdoor_loss, total_dark_loss)
    draw_accuracy_and_f1_during_training1(args, train_performs["ACC"], train_performs["F1"], test_performs["ACC"], test_performs["F1"]
                                          ,outdoor_performs["ACC"], outdoor_performs["F1"], dark_performs["ACC"], dark_performs["F1"])
    
    print(' @@ THE END @@ ')
    print(f'  > Train Best Accuracy : {np.array(train_performs["ACC"]).max():.4f}')
    print(f'  > Train Best F1-Score : {np.array(train_performs["F1"]).max():.4f}')
    print(f'  > Test Best Accuracy : {best_test_accu:.4f}')
    print(f'  > Test Epoch (Best Accuracy): {best_test_epoch}')  
    print(f'  > Test Best F1-Score : {best_test_f1:.4f}')
    print(f'  > Test Epoch (Best F1-Score): {best_test_epoch}') 
    print(f'  > Test Best CF (std: F1)') 
    print(f'  > {np.array(test_performs["Info"][best_test_epoch-1])}')
    print(f'')
    
    indoor_check_file = f"{args.save_path}/indoor.pkl"
    outdoor_check_file = f"{args.save_path}/outdoor.pkl"
    dark_check_file = f"{args.save_path}/dark.pkl"
    
    indoor_max = np.array(indoor_check["test_f1"]).max()
    outdoor_max = np.array(outdoor_check["test_f1"]).max()
    dark_max = np.array(dark_check["test_f1"]).max()
    
    indoor_epochs = []
    for idx, value in enumerate(indoor_check.values()):
        if value == indoor_max:
            indoor_epochs.append(idx+1)
    outdoor_epochs = []
    for idx, value in enumerate(outdoor_check.values()):
        if value == outdoor_max:
            outdoor_epochs.append(idx+1)
    dark_epochs = []
    for idx, value in enumerate(dark_check.values()):
        if value == dark_max:
            dark_epochs.append(idx+1)


    
    with open(indoor_check_file,'wb') as f:
        pickle.dump(indoor_check,f)
    with open(outdoor_check_file,'wb') as f:
        pickle.dump(outdoor_check,f)
    with open(dark_check_file,'wb') as f:
        pickle.dump(dark_check,f)       
        

In [6]:
args.model = 'rgbp'
args.attacktype = 'p'
args.epochs = 100 ## 1
args.cuda = 0
args

Namespace(ae_path='', attacktype='p', batchsize=4, crop=False, cuda=0, dataset=12, device='', epochs=100, inputchannel=3, lr=0.001, model='rgbp', model_path='', save_path='./hm_save_file', seed=1, skf=0, trainratio=1.0, workers=4)

In [7]:
if args.model == 'rgb':
    args.inputchannel = 3
elif args.model == 'rgbd':
    args.inputchannel = 4
elif args.model == 'rgbp':
    args.inputchannel = 6
elif args.model == 'rgbdp':
    args.inputchannel = 7
else:
    print("You need to checkout option 'model' [rgb, rgbd, rgbp, rgbpc]")
    sys.exit(0)

In [8]:
# cuda 관련 코드
use_cuda = True if torch.cuda.is_available() else False
if use_cuda : 
    if args.cuda == 0:
        args.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    elif args.cuda == 1:
        args.device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    print('device :', args.device)

device : cuda:0


In [9]:
torch.initial_seed()

1

In [10]:
train_dataset, test_dataset = load_dataset(args)
outdoor_testset = load_test_dataset(args, "2. Outdoor")
indoor_dark_testset = load_test_dataset(args, "3. Indoor_dark")

train_loader = DataLoader(train_dataset, batch_size=args.batchsize, shuffle=False, 
                          num_workers=args.workers, pin_memory=True,
                          worker_init_fn=seed_worker,generator=g)
test_loader = DataLoader(test_dataset, batch_size=args.batchsize, shuffle=False, 
                         num_workers=args.workers, pin_memory=True,
                         worker_init_fn=seed_worker,generator=g)
outdoor_loader = DataLoader(outdoor_testset, batch_size=args.batchsize, shuffle=False, 
                            num_workers=args.workers, pin_memory=True,
                            worker_init_fn=seed_worker,generator=g)
dark_loader = DataLoader(indoor_dark_testset, batch_size=args.batchsize, shuffle=False,
                         num_workers=args.workers, pin_memory=True,
                         worker_init_fn=seed_worker,generator=g)

360
120
480
480


In [ ]:
train_start = time.time()

print('Train Start !')
train(args, train_loader, test_loader, outdoor_loader, dark_loader)

train_time = str(timedelta(seconds=time.time()-train_start)).split(".")
print(f"Train Execution Time: {train_time}")  

Train Start !


[Train] Epoch[1/100][90/90] Loss:0.00308 Loss(mean):0.377712222: : 90it [00:12,  7.08it/s] 


[Train] Epoch[1/100][90/90] Loss:0.00308 Loss(mean):0.377712222
Train Accuracy : 0.8667
Train F1-score : 0.8666
  > Counter(train_labels) : Counter({1.0: 180, 0.0: 180})
  > Counter(train_preds) : Counter({1.0: 188, 0.0: 172})
 --- [Train] Confustion_Matrix & Classification_Report --- 
         Predicted:0  Predicted:1
Label:0          152           28
Label:1           20          160
              precision    recall  f1-score   support

         0.0       0.88      0.84      0.86       180
         1.0       0.85      0.89      0.87       180

    accuracy                           0.87       360
   macro avg       0.87      0.87      0.87       360
weighted avg       0.87      0.87      0.87       360



[Test] Epoch[1/100][30/30] Loss:0.22637 Loss(mean):0.23851: : 30it [00:01, 17.70it/s]
[Test] Epoch[1/100][120/30] Loss:0.02705 Loss(mean):0.44708: : 120it [00:11, 10.08it/s]
[Test] Epoch[1/100][120/30] Loss:3.39183 Loss(mean):1.87055: : 120it [00:10, 11.11it/s]


[Test] Epoch[1/100][120/30] Loss:3.39183 Loss(mean):0.23851
Test Accuracy : 0.9083
Test F1-score : 0.9076
 --- [Test] Confustion_Matrix & Classification_Report --- 
         Predicted:0  Predicted:1
Label:0           60            0
Label:1           11           49
              precision    recall  f1-score   support

         0.0       0.85      1.00      0.92        60
         1.0       1.00      0.82      0.90        60

    accuracy                           0.91       120
   macro avg       0.92      0.91      0.91       120
weighted avg       0.92      0.91      0.91       120

 @@ New Save Situation !! @@ 
 @@ New Current Epoch : 1
 
 @@ New Train Accuracy : 0.8667
 @@ New Train F1-Score : 0.8666
 
 @@ New Indoor Accuracy : 0.9083
 @@ New Indoor F1-Score : 0.9076
 
 @@ New Outdoor Accuracy : 0.6958
 @@ New Outdoor F1-Score : 0.6648
 
 @@ New Dark Accuracy : 0.6958
 @@ New Dark F1-Score : 0.6648
 
 @@ New Train Loss : 0.377712222
 @@ New Indoor Loss : 0.2385067525089653
 @@ Ne

...

[Train] Epoch[9/100][90/90] Loss:2e-05 Loss(mean):0.0006905689
Train Accuracy : 1.0000
Train F1-score : 1.0000
  > Counter(train_labels) : Counter({1.0: 180, 0.0: 180})
  > Counter(train_preds) : Counter({1.0: 180, 0.0: 180})
 --- [Train] Confustion_Matrix & Classification_Report --- 
         Predicted:0  Predicted:1
Label:0          180            0
Label:1            0          180
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       180
         1.0       1.00      1.00      1.00       180

    accuracy                           1.00       360
   macro avg       1.00      1.00      1.00       360
weighted avg       1.00      1.00      1.00       360




[Test] Epoch[9/100][30/30] Loss:5e-05 Loss(mean):6e-05: : 30it [00:01, 22.50it/s]   
[Test] Epoch[9/100][120/30] Loss:0.14571 Loss(mean):0.64905: : 120it [00:04, 24.50it/s]
[Test] Epoch[9/100][120/30] Loss:0.18903 Loss(mean):0.63859: : 120it [00:09, 13.31it/s]

[Test] Epoch[9/100][120/30] Loss:0.18903 Loss(mean):6e-05
Test Accuracy : 1.0000
Test F1-score : 1.0000
 --- [Test] Confustion_Matrix & Classification_Report --- 
         Predicted:0  Predicted:1
Label:0           60            0
Label:1            0           60
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        60
         1.0       1.00      1.00      1.00        60

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120

 @@ New Save Situation !! @@ 
 @@ New Current Epoch : 9
 
 @@ New Train Accuracy : 1.0000
 @@ New Train F1-Score : 1.0000
 
 @@ New Indoor Accuracy : 1.0000
 @@ New Indoor F1-Score : 1.0000
 
 @@ New Outdoor Accuracy : 0.7562
 @@ New Outdoor F1-Score : 0.7467
 
 @@ New Dark Accuracy : 0.7562
 @@ New Dark F1-Score : 0.7467
 
 @@ New Train Loss : 0.0006905689
 @@ New Indoor Loss : 6.207663379124521e-05
 @@ 


[Train] Epoch[10/100][90/90] Loss:2e-05 Loss(mean):0.0005574293: : 90it [00:08, 10.35it/s]  

[Train] Epoch[10/100][90/90] Loss:2e-05 Loss(mean):0.0005574293
Train Accuracy : 1.0000
Train F1-score : 1.0000
  > Counter(train_labels) : Counter({1.0: 180, 0.0: 180})
  > Counter(train_preds) : Counter({1.0: 180, 0.0: 180})
 --- [Train] Confustion_Matrix & Classification_Report --- 
         Predicted:0  Predicted:1
Label:0          180            0
Label:1            0          180
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       180
         1.0       1.00      1.00      1.00       180

    accuracy                           1.00       360
   macro avg       1.00      1.00      1.00       360
weighted avg       1.00      1.00      1.00       360



In [ ]:
print('Done')

In [ ]:
check_model

In [ ]:
check_model.eval()
print(check_model.bn1.running_mean)
print(check_model.bn1.running_var)

In [ ]:
for name, param in check_model.named_parameters():
    print(name, param.data)

In [ ]:
for name, param in check_model.fc.named_parameters():
    print(name, param.data)

In [ ]:
for i in train_loader :
    break
print(i[3])

In [ ]:
len(i)

In [ ]:
i[3]

In [ ]:
i[0].shape

In [ ]:
plt.imshow(i[0][0].permute(1,2,0))
plt.show()

In [ ]:
i[0][0].permute(1,2,0).shape

In [ ]:
i[0][3].permute(1,2,0)

In [ ]:
args.seed